In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import requests
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from matplotlib import pyplot


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SHASHWAT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv("D:/Recommender System Datasets/lthing_data/reviews2.csv")
df=df[1:]
df = df.reset_index()

In [4]:
df

,index,Unnamed: 0,comment,nhelpful,unixtime,work,flags,user,stars,time
0,1,1,"Every evening, the brave queen of Persia, Shah...",0.0,1.332288e+09,73960.0,[],Elizabeth.Wong98,4.5,"Mar 21, 2012"
1,2,2,Its the 1983-84 school year and Edward Zanni o...,0.0,1.253405e+09,69413.0,[],rivkat,3.0,"Sep 20, 2009"
2,3,3,"I became privy to this book, when another auth...",0.0,1.299888e+09,9523995.0,[],suz.haugland,4.0,"Mar 12, 2011"
3,4,4,.,0.0,1.327277e+09,368228.0,[],amoskovacs,4.0,"Jan 23, 2012"
4,5,5,This is a tremendous story of a freed woman an...,0.0,1.329005e+09,11243828.0,[],CandyH,4.0,"Feb 12, 2012"
...,...,...,...,...,...,...,...,...,...,...
995,996,996,With an abundance of excellent books about the...,1.0,1.219882e+09,5474830.0,[],khuggard,2.0,"Aug 28, 2008"
996,997,997,http://www.imdb.com/title/tt71853/ http://www....,0.0,1.280794e+09,8015336.0,"['abuse', 'not_a_review']",WilliamHartPhD,NaN,"Aug 3, 2010"
997,998,998,"Carley DeCarlo, a financial advisor newspaper ...",0.0,1.154909e+09,31799.0,[],dutchmarbel,NaN,"Aug 7, 2006"
998,999,999,In the various discussions/writings about the ...,0.0,1.286410e+09,245554.0,[],cyderry,2.5,"Oct 7, 2010"


In [5]:
def _removeNonAscii(s):
    s=str(s)
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    text=str(text)
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df['cleaned'] = df['comment'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)

In [6]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())


In [103]:
model = Word2Vec(window = 10, sg = 1, hs = 0, negative = 10, alpha=0.03, min_alpha=0.0007, seed = 14)
model.build_vocab(corpus, progress_per=200)

model.train(corpus, total_examples = model.corpus_count, epochs=10, report_delay=1)


(561463, 818240)

In [8]:
watch = df[["work", "comment"]]
watch

,work,comment
0,73960.0,"Every evening, the brave queen of Persia, Shah..."
1,69413.0,Its the 1983-84 school year and Edward Zanni o...
2,9523995.0,"I became privy to this book, when another auth..."
3,368228.0,.
4,11243828.0,This is a tremendous story of a freed woman an...
...,...,...
995,5474830.0,With an abundance of excellent books about the...
996,8015336.0,http://www.imdb.com/title/tt71853/ http://www....
997,31799.0,"Carley DeCarlo, a financial advisor newspaper ..."
998,245554.0,In the various discussions/writings about the ...


In [9]:
watch_dict = watch.groupby('work')['comment'].apply(list).to_dict()


In [104]:
ms = model.wv.similar_by_vector('every', topn= 6)[0:]
ms

[('single', 0.7657905220985413),
 ('drive', 0.725574791431427),
 ('sentence', 0.7208585739135742),
 ('sitting', 0.7207288146018982),
 ('dinner', 0.7107368111610413),
 ('hold', 0.7059918642044067)]

In [11]:
def vectors(x):
    global word_embeddings
    word_embeddings = []
    arr=[]
    for index, row in df.iterrows():
        line=row['cleaned']
        avgword2vec = None
        count = 0
        """
        for word in model.wv.key_to_index:
            if(word not in line.split()):
                arr.append(0)
            else:
                arr.append(model.wv.key_to_index[word])
        """
        
        for word in line.split():
            if word in model.wv:
                count += 1
                if avgword2vec is None:
                    avgword2vec = model.wv[word]
                else:
                    avgword2vec = avgword2vec + model.wv[word]
        
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        word_embeddings.append([row['work'],avgword2vec])

In [12]:
vectors(df)
len(word_embeddings)

1000

In [13]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 5, stop_words='english')
tfidf.fit(df['cleaned'])

tfidf_list = dict(zip(tfidf.get_feature_names_out(), list(tfidf.idf_)))
tfidf_feature = tfidf.get_feature_names_out() 

In [97]:
tfidf_list

{'04': 5.962844630259907,
 '09': 6.116995310087166,
 '10': 5.018383021419056,
 '100': 5.711530201979001,
 '11': 5.269697449699962,
 '12': 4.864232341591798,
 '13': 5.962844630259907,
 '14': 5.711530201979001,
 '15': 5.6061696863211745,
 '16': 5.829313237635384,
 '17': 5.711530201979001,
 '18': 5.51085950651685,
 '19': 5.962844630259907,
 '1970': 6.116995310087166,
 '1984': 6.116995310087166,
 '20': 5.711530201979001,
 '200': 5.962844630259907,
 '2004': 5.711530201979001,
 '2006': 5.962844630259907,
 '2007': 5.711530201979001,
 '2008': 5.4238481295272205,
 '2009': 6.116995310087166,
 '2011': 5.51085950651685,
 '2012': 5.51085950651685,
 '20th': 5.829313237635384,
 '20th century': 5.829313237635384,
 '24': 6.116995310087166,
 '25': 6.116995310087166,
 '30': 5.962844630259907,
 '3rd': 6.116995310087166,
 '50': 5.6061696863211745,
 'abilities': 5.829313237635384,
 'ability': 5.075541435259004,
 'able': 4.1475546636216585,
 'absent': 6.116995310087166,
 'absolute': 6.116995310087166,
 'abso

In [86]:
tfidf_vectors = []; 
line = 0;
c=0

for desc in corpus: 
    sent_vec = np.zeros(100) 
    weight_sum =0; 
    for word in desc: 
        if word in model.wv and word in tfidf_feature:
            vec = model.wv[word]
            tf_idf = tfidf_list[word] * (desc.count(word) / len(desc))
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    #sent_vec=np.dot(sent_vec,float(df[c:c+1]["stars"])/5)
    tfidf_vectors.append(sent_vec)
    line += 1
    c=c+1

In [98]:
tfidf_vectors

[array([-0.02419778,  0.0820615 ,  0.1048987 ,  0.05724022,  0.22229564,
        -0.10858466,  0.16169937,  0.04540742,  0.05167856,  0.02917509,
        -0.32283373, -0.02603724, -0.027695  , -0.03064279, -0.57451981,
         0.03721351, -0.19407264,  0.24716105,  0.17227453,  0.03731796,
         0.07739255, -0.20812619,  0.198275  ,  0.14773469, -0.26464022,
        -0.28075976, -0.0830938 ,  0.15795624,  0.42113129, -0.03349357,
        -0.00150575,  0.07740681,  0.09985931,  0.03108454,  0.23057189,
         0.20882545,  0.03971602, -0.13095679, -0.01838397, -0.10763354,
         0.10939847,  0.09641492,  0.20546883,  0.19650274,  0.06925948,
        -0.14461676, -0.30808   , -0.01209783,  0.19643238,  0.28242682,
        -0.27750834, -0.23519288, -0.04221898, -0.23154372, -0.11167482,
        -0.22993792, -0.04657145, -0.07625665,  0.10587085,  0.05288723,
         0.00313968,  0.11991238, -0.2683642 , -0.04129087, -0.31937487,
        -0.3887547 , -0.18685741,  0.11024895, -0.0

In [99]:
def recommendations(cleaned):
    cosine_similarities = cosine_similarity(tfidf_vectors,  tfidf_vectors)
    
    books = df[['work', 'cleaned', 'comment']]
    indices = pd.Series(df.index, index = df['cleaned']).drop_duplicates()
    idx = indices[cleaned]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:100]
    return sim_scores
    
        

In [100]:
recommendations(df['cleaned'][0])

[(524, 0.9666803820597226),
 (429, 0.9643284596016259),
 (411, 0.9635640529751127),
 (904, 0.9621922184909287),
 (960, 0.9609397624273543),
 (438, 0.960802071896526),
 (625, 0.9607891375510186),
 (366, 0.9603265852539025),
 (981, 0.9603212473992857),
 (232, 0.9602059537973461),
 (902, 0.9598593682777877),
 (229, 0.9587606880632118),
 (892, 0.9578490245006469),
 (143, 0.9574037353554518),
 (622, 0.9568668011731337),
 (463, 0.9561250724953486),
 (100, 0.9556942256847955),
 (928, 0.9552869224638812),
 (714, 0.9551298315676358),
 (702, 0.9543406617169973),
 (273, 0.9536883218856419),
 (151, 0.9534219577615978),
 (446, 0.9530771376872037),
 (226, 0.9530648166978081),
 (422, 0.9530462053259017),
 (779, 0.9523394975720261),
 (851, 0.9515836039117558),
 (213, 0.9513774252723098),
 (257, 0.950997780441496),
 (770, 0.9509130762183415),
 (467, 0.9505645834472212),
 (350, 0.9503689067777635),
 (530, 0.9495686708150045),
 (486, 0.949371329145752),
 (634, 0.9492591763990876),
 (849, 0.94871369146195

In [88]:
import requests
from bs4 import BeautifulSoup

def find_title(work):
    a=int(work)
    r = requests.get('https://www.librarything.com/work/'+str(a)+'/workdetails') 

    soup = BeautifulSoup(r.content, 'html.parser') 

    t=str(soup.title)
    t=t.replace("<title>Workdetails: ","")
    t=t.replace(" | LibraryThing</title>","")
    return t


In [89]:
def final_rec(user):
    books = df[['work', 'cleaned', 'comment','user']]
    dfn = df[df['user']==user]
    c=[]
    for index, row in dfn.iterrows():
        c.append(recommendations(row['cleaned']))

    k=[]
    for i in c:
        for j in i:
            k.append(j)

    k = sorted(k, key = lambda x: x[1], reverse = True)
    k = k[0:100]
    #[1,7,18,...]
    book_indices = [i[0] for i in k]
    book_indices=list(set(book_indices))
    
    
    recommend = list(books['work'].iloc[book_indices])
    r=[]
    hist=list(books[books['user']==user]['work'])
    for i in recommend:
        if(i not in hist):
            r.append(i)
        
            
    r=list(set(r))[0:5]
    for i in r:
        print(find_title(i))
        print(i)



In [91]:
final_rec('_Zoe_')

The Late, Lamented Molly Marx by Sally Koslow
6506505.0
The Case for Christ: A Journalist's Personal Investigation of the Evidence for Jesus by Lee Strobel
3106313.0
Dissecting Death: Secrets of a Medical Examiner by Frederick T. Zugibe
308237.0
The Undomestic Goddess by Sophie Kinsella
8205.0
Blindsight by Peter Watts
1333265.0
